<a href="https://colab.research.google.com/github/karl-karlsson/notebooks/blob/main/laptops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can open a copy of this notebook in Google Colab by clicking the icon above

-----

**This project is ongoing, ie. not finished**

![laptop](https://freesvg.org/img/1520464168.png)

# Building fast queries on a CSV

This was originally a guided project from Dataquest ([link](https://www.dataquest.io/)). 

I've downloaded a dataset on laptops to my GitHub respository from Kaggle ([info about dataset](https://www.kaggle.com/datasets/ionaskel/laptop-prices)).

I wanted to do the project in Google Colab (rather than on the Dataquest platform) so I've had to make some adjustments with regards to loading in the data. Other than that, however, this is very much a *guided* project. So I don't want to take credit for solutions etc. I've done the project for my own learning experience.

I'm however glad i decided to download the data myself. Along the way I´ve run in to some issues (such as the need to include `replace('"','')` when reading in the file to avoid problems later. Learning by trial and error is the best!


# Scenario
Let's imagine that we own an online laptop store and want to build a way to answer a few different business questions about our inventory. The `laptops.csv` file is our inventory.

The goal of the project is to create a new class to represent our inventory. The class will automatically load in the .csv file from the web. I'll also define methods that belongs to the new class. These methods will help us answer questions such as:

* Given a laptop id, find the corresponding data.
* Given an amount of money, find whether there are two laptops whose total price is that given amount.
* Identify all laptops whose price falls within a given budget.

In the first step I'll:
* Download the file from GitHub using the `requests` package
* Read in the file using the `csv` package 
* Print out the column names and the first five rows 



In [84]:
import csv
import requests

url = 'https://raw.githubusercontent.com/karl-karlsson/data/main/laptops.csv'


with requests.Session() as s:
    download = s.get(url)

    decoded_content = download.content.decode('latin-1')

    csv_read_file = csv.reader(decoded_content.replace('"','').splitlines())
    a_list = list(csv_read_file)

print("Column names")
print(a_list[0])
print("\n")
print("Inventory")
for row in range(1,6):
  print(a_list[row])


Column names
['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros']


Inventory
['1', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339.69']
['2', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898.94']
['3', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575.00']
['4', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537.45']
['5', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB',

I'll start by doing the following:

* Create a new `Inventory` class that takes an url as input and reads in a .csv file
* Converts `Price_euros` to integer
* Define three methods:
  * `attributes` that prints out the column names
  * `stock` that returns the current number of laptops (all rows exept column names) in stock
  * `data` that returns all rows, except the column names

I will build out the `Inventory` class with more methods for more advanced operations further down

In [85]:
class Inventory():                    
    
    def __init__(self, url): 
      with requests.Session() as s:
          download = s.get(url)

          decoded_content = download.content.decode('latin-1')

          csv_read_file = csv.reader(decoded_content.splitlines(), delimiter=',')
          data = list(csv_read_file)

          self.attributes = data[:1]
          self.data = data[1:]
          self.stock = len(data[1:])
                  


Let's create a new `Inventory` instance named `laptops` from the url used before.

Let´s print out the attributes of laptops by calling the `laptops.attributes` method on `laptops`


In [86]:
laptops_url = 'https://raw.githubusercontent.com/karl-karlsson/data/main/laptops.csv' 

laptops = Inventory(laptops_url)  

print(laptops.attributes)

[['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros']]


We can check how many laptops are currently in stock by calling the `stock` method on `laptops`

In [87]:
print("Current stock:", 
      laptops.stock, 
      "laptops")

Current stock: 1303 laptops


Next I'll define a new method (`get_id`) that takes an id as input and returns the corresponding laptop. This could be convenient if a customer customer comes to our store with a purchase slip because then we can quickly identify the laptop to which it corresponds.

I´ll also convert `Price_euros` to integer (I´ll use that later)


In [99]:
class Inventory():                    
    
    def __init__(self, url): 
      with requests.Session() as s:
          download = s.get(url)

          decoded_content = download.content.decode('latin-1')

          csv_read_file = csv.reader(decoded_content.replace('"','').splitlines()) # Needed to include this! Some lines wheren't reading properly...
          data = list(csv_read_file)
          self.attributes = data[:1]
          self.data = data[1:]
          for row in self.data:
            row[0] = int(row[0]) # Convert ID to integer
          for row in self.data:              
            row[-1] = int(float(row[-1])) # Convert price_euros to integer
          self.stock = len(data[1:])

    def get_id(self, laptop_id):  
        for row in self.data:                  
            if row[0] == laptop_id:
                return row
        return None                           

Let's see if we can load the inventory data (as an `Inventory` object) and use the new `get_id` method.

In [ ]:
# Reload the data
laptops = Inventory(laptops_url)

# Get laptop with ID = 25
laptops.get_id(25)

[215,
 'Acer',
 'Aspire 7',
 'Notebook',
 '15.6',
 'Full HD 1920x1080',
 'Intel Core i7 7700HQ 2.8GHz',
 '8GB',
 '1TB HDD',
 'Nvidia GeForce GTX 1050',
 'Linux',
 '2.4kg',
 779]

In [ ]:
# Get laptop with ID = 215
laptops.get_id(215)